# Solving Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai


**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# OpenAI communication

from openai import OpenAI
import os
import getpass


os.environ['OPENAI_API_KEY'] = getpass.getpass('Paste API key:')
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY']
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)
OpenAI()
print(completion.choices[0].message)

In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '138liOBnXQvw28By66Ahd5ArOYDg_LOpKPaR_1LbdkUg'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

# Task 1. Classify emails

In [ ]:
# Mounts Google Drive to access files stored in a personal drive.
# Note: This is configured for a personal Google Drive setup and is intended for development and testing.
# For production, replace this setup with a more secure and scalable file storage solution, such as a cloud storage service
# (e.g., Google Cloud Storage, AWS S3) that supports access control and scalability.

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Crossover Assessments'

In [ ]:
# Install and import necessary libraries for Google Sheets integration, language model processing, and vector storage.


%pip install gspread oauth2client
%pip install langchain
%pip install -U langchain langchain-openai langchain-community
%pip install faiss-cpu

# Import libraries for Google Sheets API access, authentication, and language model processing
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

# Import LangChain components for creating and managing LLM-based chains and embeddings
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
import json
import ast

In [ ]:
# SET UP OBSERVABILITY FOR LANGCHAIN
# This setup enables tracing and monitoring for LangChain operations, helping to track usage, performance, and errors.
# Environment variables are used to configure observability with LangChain's tracing API.
#
# Note: The default endpoint (https://api.smith.langchain.com) is a LangChain-provided API.
# For production use, you should implement and configure your own observability solution or API endpoint
# to meet specific logging, security, and compliance requirements within your organization.

# Enable advanced tracing to capture detailed operation metrics
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# Set the observability API endpoint (replace this with your own in production)
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# Prompt for an API key securely
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass('Paste API key:')
# Set the project name for organized tracking within the observability platform
os.environ['LANGCHAIN_PROJECT'] = 'crossover-assessment'



Paste API key:··········


In [ ]:
# Create a vector store for the product catalog.
# For now, an in-memory vector store (FAISS) is used to quickly retrieve similar products based on embeddings.
# In a production setting, consider using a scalable, persistent backend like Chroma or Pinecone to handle
# larger datasets and support distributed access.

# Load the product DataFrame into LangChain's document format
# - Using "name" as the main content column to create embeddings based on product names
product_loader = DataFrameLoader(products_df, page_content_column="name")
product_documents = product_loader.load() # Load data into document format for embedding

# Generate embeddings for product documents and initialize FAISS vector store for similarity search
# Use OpenAI's embedding model to create product embeddings
embeddings = OpenAIEmbeddings()
product_vectors = FAISS.from_documents(product_documents, embeddings)


In [ ]:
# Create prompt templates for classification and response generation
# Each prompt template is tailored for a specific task: classification, order processing, order extraction, and product inquiry.
# Prompts include detailed instructions to guide the assistant's responses, ensuring accuracy, professionalism, and alignment with the tone of customer interactions.


classification_prompt = PromptTemplate(
    input_variables=["email"],
    template="""
    You are an expert customer service assistant.
    Classify the following email into one of these categories:
    - "product inquiry" if the email is asking for more information about a product or any time the email does not specify a product name or quantity. Use this exact phrase when classifying.
    - "order request" if the email is requesting to buy or order a product. Use this exact phrase when classifying.
    Read both the subject and the message of the email to make the classification. Not every email with a product id is an order request.
    Return only the category and no other text. Note that some of the emails are not in English. Identify the language of the email and translate it to English before classifying.

    Email: {email}

    """
)


order_prompt = PromptTemplate(
    input_variables=["product_info", "email"],
    template="""

    Your name is AImy. You are a helpful, professional and friendly assistant that responds to customer emails.
    Respond to the order request based on product info. Check 'stock'. Check if there is enough stock to fulfill the order. If there is enough stock, fill the order.
    If there is not enough stock, suggest similar alternatives or suggest the customer waits for a restock. If there is some but not enough stock,
    fill the order and suggest similar alternatives or suggest the customer waits for a restock for the remaining items. Give examples of similar items from the product list.
    Only use products from the product list. Do not make up any products. Always double-check the stock numbers to ensure you are not making a mistake.
    Think about each step before you take it. If you do not understand an order, say you don't understand and ask for clarification.
    If no products are specified, respond precisely with "I am unable to process this order. Please provide more details".
    Never start an email with "I'm sorry..."
    Assume we can place the order once product and quantity is verified. Shipping details can be collected in a later step.
    Please note that some of the emails are not in English. Identify the language of the email and respond in the same language.
    Make sure all responses are properly formatted and match the tone of the order email.


    Product Info: {product_info}
    Email: {email}

    """
)

order_extraction_prompt = PromptTemplate(
    input_variables=["product_info", "email"],
    template="""
    You are an expert customer service assistant.
    Extract each product name and its requested quantity from the following email.
    If a quantity is not specified, assume a quantity of 1.
    Only return a string in the following format:
    {{'product_id': 'CLF2109', 'product_name': 'Cable Knit Beanies', 'quantity': 2}}

    Do not return any other text.

    Email: {email}

    """
)


inquiry_prompt = PromptTemplate(
    input_variables=["product_info", "email"],
    template="""

    Your name is AImy.You are a helpful, professional and friendly assistant that responds to customer emails.
    Answer the email based on the product info below.  If it is a product inquiry, answer the question.
    If you can't answer the question, ask for more information. Do not make anything up.
    Please note that some of the emails are not in English. Identify the language of the email and respond in the same language.
    Make sure all responses are properly formatted and match the tone of the inquiry.


    Email: {email}

    """
)


In [ ]:
# Initialize the language models (LLMs) for processing responses
# - Using OpenAI's gpt-4o for initial response generation and email classification tasks.
# - Using OpenAI's gpt-4 for order processing tasks where handling missing or incomplete inputs is crucial.
#
# Rationale:
# Through experimentation, I observed that gpt-4o struggles with scenarios where expected inputs are partially missing or incomplete.
# In contrast, gpt-4 demonstrates better robustness and adaptability, making it more suitable for handling order processing.
# Temperature is set to 0 for both models to ensure deterministic, consistent responses.

mdl = ChatOpenAI(model_name='gpt-4o', temperature=0)  # Model for classification and general response generation
mdl4 = ChatOpenAI(model_name='gpt-4', temperature=0)  # Model for order processing


In [ ]:
# Set up processing chains with LangChain to manage classification, order handling, and response generation.
# Each chain is tailored to specific tasks, leveraging the strengths of different models:
# - gpt-4o is used for general formatting and extraction tasks, as it produces well-structured outputs for classifications and inquiries.
# - gpt-4 is used for order processing tasks due to its superior handling of missing or incomplete data.
#
# A parser (StrOutputParser) is applied to each chain to standardize the output for consistent downstream handling.

parser = StrOutputParser()  # Ensures that chain outputs are returned in a uniform string format

# Chains for each specific task:
classification_chain = classification_prompt | mdl | parser  # Classifies email intent as "order request" or "product inquiry"
order_chain = order_prompt | mdl4 | parser  # Processes order requests, handling complex data dependencies more effectively
order_extraction_chain = order_extraction_prompt | mdl | parser  # Extracts product details and quantities from the email content
inquiry_chain = inquiry_prompt | mdl | parser  # Handles product inquiries, generating informative responses based on relevant product data


In [ ]:
# Initialize Google Sheets API client and access the target document on Google Drive
# This setup currently accesses a personal Drive using credentials stored in a JSON key file.
# Note: The file path and access scope may need to be adapted for production, especially to support
# multi-user or secure access in a shared or enterprise environment.


key_path = 'path/to/credentials.json'
scope = ["https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name(key_path, scope)
client = gspread.authorize(creds)

document = client.open_by_key(document_id)

In [ ]:
# Create or clear sheets to store the output results:
# 1. `email-classification`: Stores the classification of each email as either "order request" or "product inquiry".
# 2. `order-status`: Logs details for each ordered product, including product ID, quantity requested,
#    and order status (e.g., "created" or "out of stock").
# 3. `order-response`: Stores responses generated for each order request, tailored to the order details.
# 4. `inquiry-response`: Stores responses for product inquiries, providing relevant information based on customer questions.

# Attempt to access each sheet by name; if not found, create it.
# If the sheet exists, clear previous content to prepare for new data.
try:
    email_classification_sheet = document.worksheet('email-classification')
    email_classification_sheet.clear()
except gspread.exceptions.WorksheetNotFound:
    email_classification_sheet = document.add_worksheet(title="email-classification", rows=len(emails_df)+1, cols=2)

try:
    order_status_sheet = document.worksheet('order-status')
    order_status_sheet.clear()
except gspread.exceptions.WorksheetNotFound:
    order_status_sheet = document.add_worksheet(title="order-status", rows=len(emails_df)+1, cols=4)

try:
    order_response_sheet = document.worksheet('order-response')
    order_response_sheet.clear()
except gspread.exceptions.WorksheetNotFound:
    order_response_sheet = document.add_worksheet(title="order-response", rows=len(emails_df)+1, cols=2)

try:
    inquiry_response_sheet = document.worksheet('inquiry-response')
    inquiry_response_sheet.clear()
except gspread.exceptions.WorksheetNotFound:
    inquiry_response_sheet = document.add_worksheet(title="inquiry-response", rows=len(emails_df)+1, cols=2)

In [ ]:
#List to store classification results
classification_results = []

# Task 2. Process order requests

In [ ]:
#Lists to store order status, order responses, inquiry responses
order_status_results = []
order_response_results = []
inquiry_response_results = []

# Task 3. Handle product inquiry

In [ ]:
# This loop forms the core of the email processing pipeline, handling each email in the dataset.
# It begins by classifying each email's intent (order request or product inquiry) using the classification chain.
# For order requests, the code extracts individual product entries and their quantities from the email,
# then performs a similarity search to identify matching products in the catalog.
# For each product, it checks stock availability and assigns an order status ("created" or "out of stock")
# based on whether the requested quantity is in stock.
# It then generates and records a customized response for each product line in the order,
# updating stock levels and saving the order status for further processing.
#
# For product inquiries, the code retrieves the most relevant products via similarity search and generates a
# tailored response that addresses the customer's query, leveraging product details to provide accurate and engaging
# information. All generated responses, whether for order requests or inquiries, are appended to their respective
# results lists, preparing them for final storage or dispatch.


for _, row in emails_df.iterrows():
    email_id = row['email_id']  # Extract the unique email ID
    email_message = row['message']  # Extract the email content for processing
    # Classify the email as either an "order request" or "product inquiry" using the classification chain
    classification = classification_chain.invoke({"email": email_message})
    print(f"Email ID: {email_id}, Classification: {classification}")
    # Append classification result to the classification_results list
    classification_results.append({"email ID": email_id, "category": classification})
    products_ordered = []
    # Process order requests
    if classification == "order request":
        product_entries = order_extraction_chain.invoke({"email": email_message}).split('\n')
        for entry in product_entries:
          products_ordered.append(ast.literal_eval(entry))
          products_ordered_df = pd.DataFrame(products_ordered)
          product_name = str(products_ordered_df['product_name'].iloc[-1])
          quantity_requested = products_ordered_df['quantity'].iloc[-1]
          #print(product_name, quantity_requested)

          # Use vector search to retrieve products that are relevant to the email content
          relevant_products = product_vectors.similarity_search(product_name,k=1)[0]
          product_id = relevant_products.metadata["product_id"]

          # Check the stock level of the identified product in the products DataFrame
          stock = products_df.loc[products_df['product_id'] == product_id, 'stock'].values[0]

          # Determine order status based on stock availability
          if stock >= quantity_requested:
              status = "created"  # Order can be fulfilled
              # Decrement stock to reflect the new level after order fulfillment
              products_df.loc[products_df['product_id'] == product_id, 'stock'] -= quantity_requested
          else:
              status = "out of stock"  # Insufficient stock to fulfill the order

          # Append order status to order_status_results list for each product in the order
          order_status_results.append({
              "email ID": email_id,
              "product ID": product_id,
              "quantity": quantity_requested,
              "status": status
          })

          # Generate a response to the order request based on order status
          try:
            response =  order_chain.invoke({
              "email": email_message,
              "product_info": relevant_products.page_content
            })
          except Exception as e:
            response = str("I'm unable to process and respond to this email at this time. Please try again later.")
          # Append the response to the order_response_results list
          order_response_results.append({"email ID": email_id, "response": response})
          print(f"Response: {response}")

    # Process product inquiries
    elif classification == "product inquiry":
        # Retrieve relevant products using vector search to provide accurate response content
        relevant_products = product_vectors.similarity_search(email_message,k=2)
        # Generate a response to the inquiry based on the most relevant product found
        try:
          response =  inquiry_chain.invoke({
            "email": email_message,
            "product_info": relevant_products[0].page_content  # Provide info on the most relevant product
          })
        except Exception as e:
          response = str("I'm unable to process and respond to this email at this time. Please try again later.")

        # Append the inquiry response to inquiry_response_results list
        inquiry_response_results.append({"email ID": email_id, "response": response})
        print(f"Response: {response}")

Email ID: E001, Classification: order request
Leather Bifold Wallets 1
Response: Dear Customer,

Thank you for your interest in our LTH0976 Leather Bifold Wallets. I am pleased to inform you that we currently have a sufficient stock to fulfill your order. I will proceed with placing your order for all the remaining wallets.

Once your order is confirmed, we will send you an email with the shipping details and expected delivery date. 

Thank you for choosing us for your boutique shop. We are confident that these wallets will be a great addition to your inventory.

Best Regards,
AImy
Email ID: E002, Classification: order request
Vibrant Tote bag 1
Response: Dear Jessica,

Thank you for your interest in our VBT2345 Vibrant Tote bag. I completely agree with you, tote bags are indeed very convenient and versatile. 

I am pleased to inform you that we currently have the Vibrant Tote in stock and are ready to process your order. 

Please let us know the quantity you wish to order and we will 

In [ ]:
# Update Sheets
# Populate the classification sheet with the classification results
email_classification_sheet.update([["email ID", "category"]] + [[result["email ID"], result["category"]] for result in classification_results])
# Populate the order status sheet with order status results
order_status_sheet.update([["email ID", "product ID", "quantity", "status"]] + [[result["email ID"], result["product ID"], result["quantity"].item(), result["status"]] for result in order_status_results])
# Update stock levels
products_sheet = document.worksheet('products')
products_sheet.clear()
products_sheet.update([products_df.columns.values.tolist()] + products_df.values.tolist())
# Populate the order response sheet with generated order responses
order_response_sheet.update([["email ID", "response"]] + [[result["email ID"], result["response"]] for result in order_response_results])
# Populate the inquiry response sheet with generated responses to product inquiries
inquiry_response_sheet.update([["email ID", "response"]] + [[result["email ID"], result["response"]] for result in inquiry_response_results])

{'spreadsheetId': '138liOBnXQvw28By66Ahd5ArOYDg_LOpKPaR_1LbdkUg',
 'updatedRange': 'products!A1:G100',
 'updatedRows': 100,
 'updatedColumns': 7,
 'updatedCells': 700}

In [ ]:
# The following cells are used for testing the functionality of the order extraction and inquiry response chains.
# These tests simulate sample customer emails to ensure that:
# - The `order_extraction_chain` correctly identifies and extracts product IDs, names, and quantities from an order request email.
# - The `inquiry_chain` generates an appropriate response for a vague product inquiry, prompting for more information if necessary.

# Test extraction of product details and quantities from an order request email
order_extraction_string = order_extraction_chain.invoke({"email": "Hi, this is Liz. Please send me 5 CLF2109 Cable Knit Beanies and 2 pairs of FZZ1098 Fuzzy Slippers. I'm prepping some holiday gift baskets."}).split('\n')
order_extraction_string

["{'product_id': 'CLF2109', 'product_name': 'Cable Knit Beanies', 'quantity': 5}",
 "{'product_id': 'FZZ1098', 'product_name': 'Fuzzy Slippers', 'quantity': 2}"]

In [ ]:
# Test handling of a vague product inquiry to ensure the response chain prompts for clarification if needed
inquiry_test = inquiry_chain.invoke({"email": "Hi there I want to place an order for that popular item you sell. The one that's been selling like hotcakes lately. You know what I mean right?"})
inquiry_test

'Subject: Re: Inquiry About Popular Item\n\nHi there,\n\nThank you for reaching out! We have several popular items that have been selling quickly. Could you please provide a bit more detail or specify the product name or category you are interested in? This will help me assist you better with your order.\n\nLooking forward to your response.\n\nBest regards,  \nAImy'

In [ ]:
print(order_status_results)

[{'email ID': 'E001', 'product ID': 'LTH0976', 'quantity': 1, 'status': 'created'}, {'email ID': 'E002', 'product ID': 'VBT2345', 'quantity': 1, 'status': 'created'}, {'email ID': 'E004', 'product ID': 'SFT1098', 'quantity': 3, 'status': 'created'}, {'email ID': 'E007', 'product ID': 'CLF2109', 'quantity': 5, 'status': 'out of stock'}, {'email ID': 'E007', 'product ID': 'FZZ1098', 'quantity': 2, 'status': 'created'}, {'email ID': 'E008', 'product ID': 'VSC6789', 'quantity': 1, 'status': 'created'}, {'email ID': 'E010', 'product ID': 'RSG8901', 'quantity': 1, 'status': 'created'}, {'email ID': 'E013', 'product ID': 'SLD7654', 'quantity': 1, 'status': 'created'}, {'email ID': 'E014', 'product ID': 'SWL2345', 'quantity': 1, 'status': 'created'}, {'email ID': 'E018', 'product ID': 'RSG8901', 'quantity': 2, 'status': 'out of stock'}, {'email ID': 'E019', 'product ID': 'CBT8901', 'quantity': 1, 'status': 'created'}, {'email ID': 'E019', 'product ID': 'FZZ1098', 'quantity': 1, 'status': 'out 